In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu


# Read recipe inputs
weather_alma_data2 = dataiku.Dataset("weather_alma_data2")
weather_alma_data2_df = weather_alma_data2.get_dataframe()
downtimes_dim_tp = dataiku.Dataset("downtimes_dim_tp")
downtimes_dim_tp_df = downtimes_dim_tp.get_dataframe()

In [0]:
weather_alma_data2_df.drop_duplicates(subset=['date'], inplace=True, keep='last')

In [0]:
def get_meteo_entries(meteo_data_df, start_time, end_time, downtime_id):
   tmp_df = meteo_data_df.query('@start_time < date < @end_time').copy()
   tmp_df['DOWNTIME_ENTRY_ID'] = downtime_id
   return tmp_df

In [0]:
n = 0
weather_downtimes_merged_tp = dataiku.Dataset("weather_downtimes_merged_tp")
for row in downtimes_dim_tp_df.head(1).iterrows():
    tmp_df = get_meteo_entries(weather_alma_data2_df, row[1].START_TIME, row[1].END_TIME, row[1].DOWNTIME_ENTRY_ID)
    df = downtimes_dim_tp_df.query(
        'DOWNTIME_ENTRY_ID == @row[1].DOWNTIME_ENTRY_ID'
    ).merge(
        tmp_df, on='DOWNTIME_ENTRY_ID'
    )
    if n == 0:
        weather_downtimes_merged_tp.write_with_schema(df)

with weather_downtimes_merged_tp.get_writer() as writer:
    for row in downtimes_dim_tp_df.iloc[1:].iterrows():
        tmp_df = get_meteo_entries(weather_alma_data2_df, row[1].START_TIME, row[1].END_TIME, row[1].DOWNTIME_ENTRY_ID)
        df = downtimes_dim_tp_df.query(
            'DOWNTIME_ENTRY_ID == @row[1].DOWNTIME_ENTRY_ID'
        ).merge(
            tmp_df, on='DOWNTIME_ENTRY_ID'
        )
        print(n)
        if n == 0:
            writer.write_dataframe(df)
            n += 1
        else:
            writer.write_dataframe(df)